In [ ]:
!pip install git+https://github.com/SaitejaUtpala/rieoptax.git

  Cloning https://github.com/SaitejaUtpala/rieoptax.git to /tmp/pip-req-build-8ylw217z
  Running command git clone --filter=blob:none --quiet https://github.com/SaitejaUtpala/rieoptax.git /tmp/pip-req-build-8ylw217z
  Resolved https://github.com/SaitejaUtpala/rieoptax.git to commit 1954a45e9e2b8e14307fefd1cd2ae494402cf4ab
  Preparing metadata (setup.py) ... done
  Using cached autodp-0.2.tar.gz (39 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rieoptax: filename=rieoptax-0.0.1-py3-none-any.whl size=29035 sha256=9c70712c794ed64879206bec61df9f91f4cd71e6a53f49fb821a4c8e8de5d89d
  Stored in directory: /tmp/pip-ephem-wheel-cache-rxxamq01/wheels/a7/9f/69/52ec93d81804da7aae50235c8309ff8415b9cfbf071e4d4350
  Created wheel for autodp: filename=autodp-0.2-py3-none-any.whl size=42443 sha256=576df3ffc2dbc47fde9e05ccdc1c9f6fce022101e0d3209cd5485e6c87ba5878
  Stored in directory: /home/skholkin/.cache/pip/wheels/4e/58/65/b22c7a93f9650f8c7ce140b5309a5a0563c7119ad6261b3b93
Successfu

In [1]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax import struct

import optax

import numpy as np
from flax import struct 
from clu import metrics
from dataclasses import field
from functools import partial
from typing import Any, Callable, Optional, Tuple, Union
from flax import struct  

from clu import metrics

from chex import Array
from flax import linen as nn
from flax.linen.activation import sigmoid, tanh
from flax.linen.dtypes import promote_dtype
from flax.linen.initializers import orthogonal
from flax.linen.linear import default_kernel_init
from jax import numpy as jnp
from jax import random, vmap
from chex import Array
from jax import lax
from jax.nn.initializers import Initializer as Initializer
from jax._src import dtypes

from flax.training import train_state

from rieoptax.geometry.hyperbolic import PoincareBall
key = jax.random.PRNGKey(0)


@struct.dataclass
class Metrics(metrics.Collection):
  loss: metrics.Average.from_output('loss')
    
class TrainState(train_state.TrainState):
  metrics: Metrics

/home/skholkin/projects/python_venv/lib/python3.10/site-packages/flax/core/frozen_dict.py:169: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## Poincare ball manifold

In [2]:
from rieoptax.geometry.hyperbolic import PoincareBall
dims, c = 10, -1
ball = PoincareBall(dims, c)


### Linear layer FLAX

In [3]:
dim = 10
key = jax.random.PRNGKey(0)


class RealFnEuclidian:
    def __init__(self, in_dim=10, out_dim=5):
        global key
        self.theta_vec = jax.random.normal(key, shape=(out_dim, in_dim))
        key, subkey = jax.random.split(key)
        self.theta_bias = jax.random.normal(subkey, shape=(out_dim,))
        
    def apply_fn(self, x):
        return self.theta_vec @ x + self.theta_bias
    
    
real_fn = RealFnEuclidian(in_dim=10, out_dim=5)

key, subkey = jax.random.split(key)
sampled_x = jax.random.normal(subkey, shape=(10,))
sampled_y = real_fn.apply_fn(sampled_x)


def sample_gen(dim=10): 
    global key
    key, subkey = jax.random.split(key)
    sampled_x = jax.random.normal(key, shape=(dim,))
    return sampled_x, real_fn.apply_fn(sampled_x) 
    


In [4]:
sampled_x, sampled_y, real_fn.theta_vec, real_fn.theta_bias

(Array([-0.08943313,  1.5241055 , -0.41166604,  0.8208107 ,  1.0238154 ,
         0.4196539 ,  1.0200214 ,  0.00804093,  0.63066274,  1.7859513 ],      dtype=float32),
 Array([-2.1657333, -4.6946745, -3.1985538,  3.8316498, -4.455423 ],      dtype=float32),
 Array([[-1.9496337e-01, -1.2271470e+00, -9.1779011e-01,  1.4520884e-01,
          1.8150680e-03, -7.4958706e-01, -8.0263335e-01,  1.2242906e-01,
         -1.7589016e-01,  1.0580742e+00],
        [-2.4492617e-01, -7.7644986e-01,  4.0735185e-01,  5.3473938e-01,
         -1.3593255e+00, -2.1800435e+00, -2.7196446e-01,  1.3657150e+00,
          3.0998212e-01,  3.4722403e-01],
        [-3.6875853e-01,  2.7483419e-01,  1.2167790e+00, -2.9819357e-01,
         -1.7567271e+00,  1.5572840e+00, -5.2775639e-01,  1.2997842e-01,
         -1.2879779e+00, -1.3760062e+00],
        [ 5.8405155e-01,  4.5868692e-01,  4.1681662e-02, -5.9727866e-02,
          3.0754104e-01,  1.6455851e-01, -5.2474406e-02,  1.1677581e+00,
          1.7214864e+00,  3.6793

In [5]:
class Linear(nn.Module):
  @nn.compact
  def __call__(self, x):
    print(x)
    y = nn.Dense(5)(x)
    return y

linear = Linear()
print(linear.tabulate(jax.random.PRNGKey(42), jnp.ones((1, 10))))


[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]

                               Linear Summary                                
┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path    ┃ module ┃ inputs        ┃ outputs      ┃ params                  ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         │ Linear │ float32[1,10] │ float32[1,5] │ Dense_0:                │
│         │        │               │              │   bias: float32[5]      │
│         │        │               │              │   kernel: float32[10,5] │
│         │        │               │              │                         │
│         │        │               │              │ 55 (220 B)              │
├─────────┼────────┼───────────────┼──────────────┼─────────────────────────┤
│ Dense_0 │ Dense  │ float32[1,10] │ float32[1,5] │ bias: float32[5]        │
│         │        │               │              │ kernel: float32[10,5]   │
│         │        │         

In [6]:
batch_size = 1

@struct.dataclass
class Metrics(metrics.Collection):
  loss: metrics.Average.from_output('loss')
    
from flax.training import train_state
class TrainState(train_state.TrainState):
  metrics: Metrics

def create_train_state(module, rng, learning_rate):
  """Creates an initial `TrainState`."""
  params = module.init(rng, jnp.ones([batch_size, 10]))['params'] # initialize parameters by passing a template image
  tx = optax.sgd(learning_rate)
  return TrainState.create(
      apply_fn=module.apply, params=params, tx=tx,
      metrics=Metrics.empty())

train_state = create_train_state(linear, jax.random.PRNGKey(43), 1e-2)


[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [7]:

@jax.jit
def train_step(state, batch_x, batch_y):
  """Train for a single step."""

  def loss_fn(params):
    y = state.apply_fn({'params': params}, batch_x)
    loss = ((y - batch_y) ** 2).sum()
    return loss

  grad_fn = jax.grad(loss_fn)
  grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

@jax.jit
def compute_metrics(*, state, batch_x, batch_y):
    y = state.apply_fn({'params': state.params}, batch_x)
    loss = ((y - batch_y) ** 2).mean()
    metric_updates = state.metrics.single_from_model_output(loss=loss)
    metrics = state.metrics.merge(metric_updates)
    state = state.replace(metrics=metrics)
    return state

batch_x, batch_y  = sample_gen(dim=10)

train_step(train_state, batch_x, batch_y)

train_state = compute_metrics(state=train_state, batch_x=batch_x,batch_y=batch_y)
train_state.metrics.compute()['loss']

Traced<ShapedArray(float32[10])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float32[10])>with<DynamicJaxprTrace(level=1/0)>


Array(5.6716547, dtype=float32)

In [8]:
n_iter = 1000

for i in range(n_iter):
  # Run optimization steps over training batches and compute batch metrics
    
    batch_x, batch_y  = sample_gen(dim=10)
#     print(batch_x, batch_y)
    train_state = train_step(train_state, batch_x, batch_y)
    train_state = compute_metrics(state=train_state, batch_x=batch_x,batch_y=batch_y)
    loss = train_state.metrics.compute()['loss']
    if i % 100 == 0:
        print(f'{i} Loss: {loss}')


0 Loss: 7.048534393310547
100 Loss: 1.7467708587646484
200 Loss: 0.9494543075561523
300 Loss: 0.6369355916976929
400 Loss: 0.4785136878490448
500 Loss: 0.3831923305988312
600 Loss: 0.3195391297340393
700 Loss: 0.2740207314491272
800 Loss: 0.23985354602336884
900 Loss: 0.21326224505901337


### Linear layer on Poincare Ball

In [9]:

from rieoptax.geometry.hyperbolic import PoincareBall

class NormalPoincareBall:
    def __init__(self, dim, c):
        self.dim = dim
        self.c = c
        self.ball = PoincareBall(dim, c)
        self.base_point = jnp.zeros(dim)
    
    def sample(self):
        global key
        key, subkey = jax.random.split(key)
        euclidian_sample = jax.random.normal(key, shape=self.dim)
        poincare_ball_sample = self.ball.exp(self.base_point, euclidian_sample)
        return poincare_ball_sample

    
def sample_gen(dim=10): 
    global key
    key, subkey = jax.random.split(key)
    sampled_x = jax.random.normal(key, shape=(dim,))
    return sampled_x, real_fn.apply_fn(sampled_x) 


poincare_ball_normal = NormalPoincareBall(dim=(10,), c=-1)
manifold_sample = poincare_ball_normal.sample()


In [10]:

class RealFnPoincareBall:
    def __init__(self, in_dim=10, out_dim=5, c=-1):
        self.c = c
        self.in_dim, self.out_dim = in_dim, out_dim
        global key
        key, subkey = jax.random.split(key)
        self.theta_vec = jax.random.normal(key, shape=(out_dim, in_dim))
        
        poincare_ball_normal_bias = NormalPoincareBall(dim=(out_dim,), c=-1)
        self.theta_bias = poincare_ball_normal_bias.sample()

    def apply_fn(self, x):
        ball_in = PoincareBall(self.in_dim, self.c)
        Ax = ball_in.mobius_matvec(self.theta_vec, x)
        ball_out = PoincareBall(self.out_dim, self.c)
        Ax_plus_b = ball_out.mobius_add(Ax, self.theta_bias)
        return Ax_plus_b

real_fn_poincare = RealFnPoincareBall(in_dim=10, out_dim=5)

poincare_ball_normal_x = NormalPoincareBall(dim=(10,), c=-1)

sampled_poincare = poincare_ball_normal_x.sample()
sampled_y = real_fn_poincare.apply_fn(sampled_poincare)

jnp.linalg.norm(sampled_y)

Array(0.99998385, dtype=float32)

In [11]:

in_dim, out_dim = 10, 5
c = -1

class PoincareLinear(nn.Module):
    
    param_dtype = jnp.float32
    kernel_init: Callable = default_kernel_init
    bias_init: Callable = nn.initializers.uniform(scale= 1 / jnp.sqrt(in_dim))
        
    in_dim, out_dim = in_dim, out_dim
    c = c
    
    def setup(self):
        self.scalars = self.param(
            "scalars",
            self.kernel_init,
            (self.out_dim, self.in_dim),
            self.param_dtype,
        )
    
        self.bias_poincare = self.param(
            "bias_poincare",
            self.bias_init,
            (self.out_dim,),
            self.param_dtype,
        )

    def __call__(self, x):
        ball_in = PoincareBall(self.in_dim, self.c)
        y = ball_in.mobius_matvec(self.scalars, x)
        
        ball_out = PoincareBall(self.out_dim, self.c)

        return ball_out.mobius_add(y, self.bias_poincare)


In [38]:
batch_size = 1

import flax
import numpy as np
import optax
    
from flax.training import train_state
class TrainState(train_state.TrainState):
    metrics: Metrics
    
    def apply_gradients(self, *, grads, **kwargs):
        """Updates `step`, `params`, `opt_state` and `**kwargs` in return value.

        Note that internally this function calls `.tx.update()` followed by a call
        to `optax.apply_updates()` to update `params` and `opt_state`.

        Args:
          grads: Gradients that have the same pytree structure as `.params`.
          **kwargs: Additional dataclass attributes that should be `.replace()`-ed.

        Returns:
          An updated instance of `self` with `step` incremented by one, `params`
          and `opt_state` updated by applying `grads`, and additional attributes
          replaced as specified by `kwargs`.
        """
        updates, new_opt_state = self.tx.update(
            grads, self.opt_state, self.params)
        
        
        old_bias_params = self.params['bias_poincare']
        
        ball_bias = PoincareBall(out_dim, c)
        
        bias_poincare_grads = updates['bias_poincare']
        
        r_grad_poincare_bias = ball_bias.egrad_to_rgrad(old_bias_params, bias_poincare_grads)
        
        updates = updates.unfreeze()
        updates['bias_poincare'] = r_grad_poincare_bias
        updates = flax.core.frozen_dict.freeze(updates)
        
        new_params = optax.apply_updates(self.params, updates)
        
        bias_poincare_new = new_params['bias_poincare']
        new_params = new_params.unfreeze()
        lr = self.lr
        tv = lr * r_grad_poincare_bias
        
        new_params['bias_poincare'] = ball_bias.exp(old_bias_params, tv)
        
        new_params = flax.core.frozen_dict.freeze(new_params)
        
        return self.replace(
            step=self.step + 1,
            params=new_params,
            opt_state=new_opt_state,
            **kwargs,
        )



def create_train_state(module, rng, learning_rate):
  """Creates an initial `TrainState`."""
  from flax.training import train_state
  class TrainState(train_state.TrainState):
    metrics: Metrics
    lr = learning_rate
  params = module.init(rng, jnp.ones([in_dim]))['params'] # initialize parameters by passing a template image
  tx = optax.sgd(learning_rate)
  return TrainState.create(
      apply_fn=module.apply, params=params, tx=tx,
      metrics=Metrics.empty())



In [42]:

@jax.jit
def train_step(state, batch_x, batch_y):
  """Train for a single step."""

  def loss_fn(params):
    y = state.apply_fn({'params': params}, batch_x)
    loss = ((y - batch_y) ** 2).sum()
    return loss
  
  grad_fn = jax.grad(loss_fn)
  grads = grad_fn(state.params)
  state = state.apply_gradients(grads=grads)
  return state

@jax.jit
def compute_metrics(*, state, batch_x, batch_y):
    y = state.apply_fn({'params': state.params}, batch_x)
    loss = ((y - batch_y) ** 2).mean()
    metric_updates = state.metrics.single_from_model_output(loss=loss)
    metrics = state.metrics.merge(metric_updates)
    state = state.replace(metrics=metrics)
    return state



In [43]:

# poincare_linear = PoincareLinear()
    
poincare_linear = PoincareLinear()
train_state = create_train_state(poincare_linear, jax.random.PRNGKey(43), 1e-02)

batch_x = poincare_ball_normal.sample()
batch_y = real_fn_poincare.apply_fn(batch_x)

train_state = compute_metrics(state=train_state, batch_x=batch_x, batch_y=batch_y)

biases = train_state.params['bias_poincare']
train_state = train_step(train_state, batch_x, batch_y)

biases = train_state.params['bias_poincare']

train_state = compute_metrics(state=train_state, batch_x=batch_x,batch_y=batch_y)
train_state.metrics.compute()['loss']

Array(0.6835966, dtype=float32)

In [46]:
n_iter = 1000

for i in range(n_iter):
    
    batch_x = poincare_ball_normal.sample()
    batch_y = real_fn_poincare.apply_fn(batch_x)
    
    train_state = train_step(train_state, batch_x, batch_y)
    train_state = compute_metrics(state=train_state, batch_x=batch_x,batch_y=batch_y)
    
    loss = train_state.metrics.compute()['loss']
    if i % 100 == 0:
        print(f'{i} Loss: {loss}')

0 Loss: 0.06442015618085861
100 Loss: 0.061408333480358124
200 Loss: 0.058650337159633636
300 Loss: 0.05613384395837784
400 Loss: 0.05383966118097305
500 Loss: 0.051750294864177704
600 Loss: 0.049802880734205246
700 Loss: 0.04801582545042038
800 Loss: 0.04632076248526573
900 Loss: 0.044779401272535324
